# OVITO

https://www.ovito.org/docs/current/python/introduction/overview.html <br>

Continue with Exporting data to disk

In [2]:
from ovito.io import import_file
pipeline = import_file("dump.data_gathering_T1400")

In [5]:
print(pipeline.source)

In [6]:
from ovito.modifiers import *
pipeline.modifiers.append(ColorCodingModifier(property = 'Potential Energy'))
pipeline.modifiers.append(SliceModifier(normal = (0,0,1)))

In [8]:
pipeline.modifiers.append(CommonNeighborAnalysisModifier(cutoff=3.2, only_selected=True))

In [9]:
modifier = CommonNeighborAnalysisModifier()
modifier.cutoff = 3.2
modifier.only_selected = True
pipeline.modifiers.append(modifier)

In [11]:
import ovito
print(ovito.__file__)

/jet/home/nhew/.conda/envs/ovito-env/lib/python3.10/site-packages/ovito/__init__.py


# LAMMPS Input Files

## Simulation and File Parameters

In [1]:
from pymatgen.core import Structure, Lattice, Element
from pymatgen.io.lammps.data import LammpsData
import numpy as np
import os

# Composition and simulation parameters
x = 0.5  # Cu fraction
T = 1500  # Temperature (K)
xyrate_1_per_s = 1e10  # Shear rate (1/s)
supercell_size = (14, 14, 14)
output_file = "Al1-xCux.data"
potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
#potential = "Cu1.eam.fs"

# Directory and file path formatting
xyrate_str = f"{xyrate_1_per_s:.0e}".replace("+0", "").replace("+", "")
potential_str = potential.replace("CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_", "").replace('.', '_')
path = os.path.join(
    "/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Al1-xCux",
    potential_str, f"x_{x}/{T}K/{xyrate_str}_test2/"
)
os.makedirs(path, exist_ok=True)

## Helper Functions

In [29]:
def generate_al1xcux_supercell(x, supercell_size, lattice_constant):
    lattice = Lattice.cubic(lattice_constant)
    structure = Structure.from_spacegroup(
        "Fm-3m", lattice, ["Al"], [[0, 0, 0]]
    )
    supercell = structure * supercell_size
    num_sites = len(supercell)
    num_cu = int(round(x * num_sites))
    al_indices = [i for i, site in enumerate(supercell) if site.specie == Element("Al")]
    np.random.seed(42)
    cu_indices = np.random.choice(al_indices, num_cu, replace=False)
    for idx in cu_indices:
        supercell[int(idx)] = "Cu"
    return supercell

def get_atom_types(x):
    if x == 0:
        return "Al"
    elif x == 1:
        return "Cu"
    else:
        return "Al Cu"

## Generate and Write LAMMPS Data File

In [30]:
lattice_constant = 4.0502 * (1 - x) + 3.6150 * x  # Vegard's law
supercell = generate_al1xcux_supercell(x, supercell_size, lattice_constant)
lammps_data = LammpsData.from_structure(supercell, atom_style="atomic")
lammps_data.write_file(os.path.join(path, output_file))

## Shear Strain Calculation and Simulation Time

In [31]:
# Shear Strain and Simulation Time Calculation
#
# T(t) = T0 + L0 * erate * dt
#
# Where:
#   T(t)   : Tilt factor at time t (same as delta_x)
#   T0     : Initial tilt factor
#   L0     : Original length of the box perpendicular to the shear direction
#   erate  : Shear strain rate
#   dt     : Elapsed time
#
# Example:
#   If erate = 0.01 (1/ps), then:
#     - Shear strain after 1 ps is 0.01
#     - Shear strain after 2 ps is 0.02, etc. 
# This block calculates the total simulation time and total shear strain for a given number of steps and timestep.
# You can use the results here to set step2_run_time and step3_run_time in the next block.

# Parameters
xyrate_1_per_ps = xyrate_1_per_s / 1e12  # Shear rate in 1/ps

dt = 0.001  # Timestep in ps (1 fs)

step2_run_time = int(1 / xyrate_1_per_ps * 1000)  # Step 2: NEMD equilibration (timesteps yielding 1 L0)
step3_run_time = int(5 / xyrate_1_per_ps * 1000)  # Step 3: Data gathering (timesteps yielding 5 L0)

## Generate and Write LAMMPS Input Script

In [33]:
atom_types = get_atom_types(x)
pair_coeff = "eam/alloy"
#pair_coeff = "eam/fs"

step1_run_time = 1_000_000 # Step 1: Temperature equilibration (1 ns)
num_bins = 28 

in_nemd_script = f"""# Sample LAMMPS input script for viscosity of FCC solid
# NEMD via fix deform and fix nvt/sllod

# Settings
variable        T equal {T}
variable        P equal 1 

variable        xyrate_1_per_s equal {xyrate_1_per_s}
variable        xyrate_1_per_ps equal v_xyrate_1_per_s/1.0e12

# Problem setup
units		    metal
dimension	    3
boundary        p p p
atom_style	    atomic

read_data	    Al1-xCux.data
change_box      all triclinic

pair_style      {pair_coeff}
pair_coeff      * * {potential} {atom_types}

# --- Step 1. Temperature equilibration ---
velocity        all create $T 97287
fix             1 all npt temp $T $T $(100.0*dt) iso $P $P $(1000.0*dt)

thermo          1000
#dump            equilibration all custom 50 dump.equilibration_T{T} id type x y z vx vy vz 
run	            {step1_run_time}

#undump         equilibration
unfix  	        1
reset_timestep  0

# --- Step 2. Turn on NEMD shear and equilibrate some more ---
#variable	    xyrate equal ${{srate}}/ly # Shear rate defined relative to perpendicular dimension (1/ps)

fix		        1 all nvt/sllod temp $T $T $(100.0*dt)

# Perform deformation every 1 timestep
# erate - engineering shear strain rate (1/time units)
fix		        2 all deform 1 xy erate ${{xyrate_1_per_ps}} remap v

compute		    usual all temp
compute		    tilt all temp/deform

thermo          1000
thermo_style	custom step temp c_usual epair etotal press pxy
thermo_modify	temp tilt

#dump            nemd_equilibration all custom 50 dump.nemd_equilibration_T{T} id type x y z vx vy vz
run		        {step2_run_time}
#undump          nemd_equilibration
reset_timestep  0

# --- Step 3. Data gathering run ---
# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# For this case, there seems to be no difference between lower, center, and upper options.
compute         layers_center all chunk/atom bin/1d y center {1/num_bins} units reduced
fix		        3 all ave/chunk 10 {int(step3_run_time/10)} {step3_run_time} layers_center vx file profile_center_T{T}.nemd

# This only takes into account the top and bottom velocity points to fit the linear profile
# It doesn't even use fix 4, which is the only one that actually computes the profile
# The viscosity is calculated from the slope of the linear profile
variable        timestep equal step
variable	    visc equal -pxy/(v_xyrate_1_per_ps)*0.0001 # Computed every timestep. Convert bar.ps to mPa.s 
variable        neg_pxy equal -pxy

# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# The running average averages the average values computed every 1000 timesteps.
fix             vprint all print 10 \"${{timestep}} ${{visc}}\" file visc_output_T{T}.txt screen no
fix             pprint all print 10 \"${{timestep}} ${{neg_pxy}}\" file pxy_output_T{T}.txt screen no
fix		        vave all ave/time 10 100 1000 v_visc ave running start 1000 file visc_run_avg_T{T}.txt
fix             pave all ave/time 10 100 1000 v_neg_pxy ave running start 1000 file pxy_run_avg_T{T}.txt

thermo          1000
thermo_style	custom step temp etotal press pxy ly v_visc f_vave
thermo_modify	temp tilt

#dump	        data_gathering all custom 100 dump.data_gathering_T{T} id type x y z
dump            id all atom 500 dump.data_gathering_T{T}

run		        {step3_run_time}
"""

with open(os.path.join(path, "in.nemd"), "w") as f:
    f.write(in_nemd_script)

## Copy Potential File

In [34]:
import shutil
src_potential = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Al1-xCux/{potential}"
dst_potential = os.path.join(path, potential)
shutil.copy(src_potential, dst_potential)

'/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Al1-xCux/Zhou04_eam_alloy/x_0.5/1500K/1e10_test2/CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy'

## Write and Submit Job Script

In [35]:
job_script = """#!/bin/bash
#SBATCH --job-name=Bridges-2
#SBATCH -A dmr190011p
#SBATCH -p RM
#SBATCH -N 2
#SBATCH --ntasks-per-node 64
#SBATCH -t 48:00:00

module purge
module load intelmpi/2021.3.0-intel2021.3.0
module load gcc/10.2.0
module load cuda/11.7.1
module load python

echo "SLURM_NTASKS: " $SLURM_NTASKS

ulimit -s unlimited
export OMP_NUM_THREADS=1

mpirun -n $SLURM_NTASKS /opt/packages/LAMMPS/lammps-23Jun2022/build/lmp -sf omp -pk omp $OMP_NUM_THREADS -in in.nemd
"""

with open(os.path.join(path, "job.sh"), "w") as f:
    f.write(job_script)

import subprocess
cwd = os.getcwd()
#os.chdir(path)
#try:
#    subprocess.run(["sbatch", "job.sh"], check=True)
#finally:
#    os.chdir(cwd)

In [51]:
import plotly.graph_objects as go

shear_strain = [1e8, 1e9, 1e10]
viscosity = [5.31635, 3.00565, 3.39067]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=shear_strain,
    y=viscosity,
    mode='markers+lines',
    marker=dict(size=10),
    name='Viscosity vs Shear Strain Rate'
))

fig.update_layout(
    title='Viscosity vs Shear Strain Rate',
    xaxis_title='Shear Strain Rate (1/s, log scale)',
    yaxis_title='Viscosity (mPa·s)',
    template='plotly_white',
    xaxis_type='log',
    width=600, 
    height=400,  
)
fig.show()

In [3]:
import numpy as np
import plotly.graph_objects as go

# Data from the chunk-averaged output
coord1 = np.array([
    0.0178571, 0.0535714, 0.0892857, 0.125, 0.160714, 0.196429, 0.232143, 0.267857, 0.303571,
    0.339286, 0.375, 0.410714, 0.446429, 0.482143, 0.517857, 0.553571, 0.589286, 0.625, 0.660714,
    0.696429, 0.732143, 0.767857, 0.803571, 0.839286, 0.875, 0.910714, 0.946429, 0.982143
])
vx = np.array([
    -0.234353, -0.228906, -0.204072, -0.18871, -0.177372, -0.162954, -0.141397, -0.124096, -0.0981958,
    -0.0798194, -0.0613376, -0.0492058, -0.0289633, -0.00140929, 0.0239957, 0.0432536, 0.0567976, 0.0646779, 0.0802907,
    0.099814, 0.11555, 0.143339, 0.167974, 0.191155, 0.218342, 0.243412, 0.263708, 0.273804
])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=vx,
    y=coord1,
    mode='lines+markers',
    name='Velocity Profile'
))
fig.update_layout(
    title='Velocity Profile at Timestep 500000',
    xaxis_title='vx',
    yaxis_title='Coord1 (y/ly)',
    template='plotly_white',
    width=600,
    height=400
)
fig.show()

In [6]:
composition = np.array([0, 0.5, 0.75, 1])
viscosity = np.array([0.6, 1.28101, 1.84, 2.6])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=composition,
    y=viscosity,
    mode='lines+markers',
    name='Viscosity vs Composition'
))
fig.update_layout(
    title='Viscosity vs Composition',
    xaxis_title='Composition (x)',
    yaxis_title='Viscosity (mPa·s)',
    template='plotly_white',
    width=600,
    height=400
)
fig.show()

# Plot the velocity profile

In [15]:
import plotly.graph_objects as go

# Function to vx for a specific timestep using Plotly
def plot_vx(timestep):
    if timestep in chunk_data:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=chunk_data[timestep]['vx'],
            y=chunk_data[timestep]['coord1'],
            mode='lines+markers',
            name=f'Timestep {timestep}'
        ))
        fig.update_layout(
            title=f'Velocity Profile at Timestep {timestep}',
            yaxis_title='y/l<sub>y</sub>',
            xaxis_title='vx',
            template='plotly_white',
            width=800,
            height=600 
        )
        fig.show()
    else:
        print(f"No data available for timestep {timestep}")

In [67]:
T = 1400
xyrate_1_per_s = 1e9

#pair_coeff = "eam/fs"
#potential = "Cu1.eam.fs"
pair_coeff = "eam/alloy"
potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{T}K/{xyrate_str}/"
velocity_profile_file = path + f'profile_center_T{T}.nemd'

# Initialize dictionaries to store the data for each chunk
chunk_data = {}

# Read the file and parse the data
with open(velocity_profile_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('#') or line.strip() == '':
            continue
        parts = line.split()
        if len(parts) == 3:
            # This is a timestep line
            current_timestep = int(parts[0])
        elif len(parts) == 4:
            # This is a data line
            chunk = int(parts[0])
            coord = float(parts[1])
            ncount = float(parts[2])
            vx = float(parts[3])
            
            if current_timestep not in chunk_data:
                chunk_data[current_timestep] = {'chunk': [], 'coord1': [], 'ncounts': [], 'vx': []}
                
            chunk_data[current_timestep]['chunk'].append(chunk)
            chunk_data[current_timestep]['coord1'].append(coord)
            chunk_data[current_timestep]['ncounts'].append(ncount)
            chunk_data[current_timestep]['vx'].append(vx)

In [60]:
time = 500000
plot_vx(time)

In [68]:
import numpy as np
import plotly.graph_objects as go

# Get all unique chunk indices (assume all timesteps have the same chunking)
first_timestep = next(iter(chunk_data))
num_chunks = len(chunk_data[first_timestep]['chunk'])
chunk_indices = np.array(chunk_data[first_timestep]['chunk'])
coords = np.array(chunk_data[first_timestep]['coord1'])

# Stack vx for each chunk across all timesteps
vx_matrix = []
for t in chunk_data:
    vx_matrix.append(chunk_data[t]['vx'])
vx_matrix = np.array(vx_matrix)  # shape: (num_timesteps, num_chunks)

# Average vx for each chunk over all timesteps
avg_vx_per_chunk = np.mean(vx_matrix, axis=0)

# Plot y/ly (coords) vs average vx
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=avg_vx_per_chunk,
    y=coords,
    mode='lines+markers',
    name='Average vx (spatial)'
))
fig.update_layout(
    title='Spatially Averaged Velocity Profile: y/ly vs. Average vx',
    yaxis_title='y / ly',
    xaxis_title='Average vx',
    template='plotly_white',
    width=800,
    height=500
)
fig.show()

In [69]:
import numpy as np
import plotly.graph_objects as go

# Get all unique chunk indices (assume all timesteps have the same chunking)
first_timestep = next(iter(chunk_data))
num_chunks = len(chunk_data[first_timestep]['chunk'])
chunk_indices = np.array(chunk_data[first_timestep]['chunk'])
coords = np.array(chunk_data[first_timestep]['coord1'])

# Stack vx for each chunk across all timesteps
vx_matrix = []
for t in chunk_data:
    vx_matrix.append(chunk_data[t]['vx'])
vx_matrix = np.array(vx_matrix)  # shape: (num_timesteps, num_chunks)

# Average vx for each chunk over all timesteps
avg_vx_per_chunk = np.mean(vx_matrix, axis=0)

# Fit a linear model: coords (y) vs avg_vx_per_chunk (x)
coeffs = np.polyfit(avg_vx_per_chunk, coords, 1)  # coeffs[0]: slope, coeffs[1]: intercept
fit_x = np.linspace(np.min(avg_vx_per_chunk), np.max(avg_vx_per_chunk), 100)
fit_y = coeffs[0] * fit_x + coeffs[1]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=avg_vx_per_chunk,
    y=coords,
    mode='lines+markers',
    name='Average vx (spatial)'
))
fig.add_trace(go.Scatter(
    x=fit_x,
    y=fit_y,
    mode='lines',
    name='Linear Fit',
    line=dict(dash='dash', color='red')
))
fig.update_layout(
    title='Spatially Averaged Velocity Profile: y/ly vs. Average vx with Linear Fit',
    yaxis_title='y / ly',
    xaxis_title='Average vx',
    template='plotly_white',
    width=800,
    height=500
)
fig.show()

print(f"Fit equation: y/ly = {coeffs[0]:.4f} * vx + {coeffs[1]:.4f}")

# Calculate and display goodness of fit (R-squared) for the linear fit
fit_y_actual = coeffs[0] * avg_vx_per_chunk + coeffs[1]
ss_res = np.sum((coords - fit_y_actual) ** 2)
ss_tot = np.sum((coords - np.mean(coords)) ** 2)
r_squared = 1 - (ss_res / ss_tot)
print(f"R-squared (goodness of fit): {r_squared:.4f}")

Fit equation: y/ly = 17.6059 * vx + 0.5856
R-squared (goodness of fit): 0.9896


In [ ]:
# Fit a linear model: coords (y) vs avg_vx_per_chunk (x)
import numpy as np
import plotly.graph_objects as go

# Fit a line: coords = slope * avg_vx_per_chunk + intercept
coeffs = np.polyfit(avg_vx_per_chunk, coords, 1)  # coeffs[0]: slope, coeffs[1]: intercept
fit_x = np.linspace(np.min(avg_vx_per_chunk), np.max(avg_vx_per_chunk), 100)
fit_y = coeffs[0] * fit_x + coeffs[1]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=avg_vx_per_chunk,
    y=coords,
    mode='lines+markers',
    name='Average vx (spatial)'
))
fig.add_trace(go.Scatter(
    x=fit_x,
    y=fit_y,
    mode='lines',
    name='Linear Fit',
    line=dict(dash='dash', color='red')
))
fig.update_layout(
    title='Spatially Averaged Velocity Profile: y/ly vs. Average vx with Linear Fit',
    yaxis_title='y / ly',
    xaxis_title='Average vx',
    template='plotly_white',
    width=800,
    height=500
)
fig.show()

print(f"Fit equation: y/ly = {coeffs[0]:.4f} * vx + {coeffs[1]:.4f}")

In [ ]:
# Calculate and display goodness of fit (R-squared) for the linear fit
fit_y_actual = coeffs[0] * avg_vx_per_chunk + coeffs[1]
ss_res = np.sum((coords - fit_y_actual) ** 2)
ss_tot = np.sum((coords - np.mean(coords)) ** 2)
r_squared = 1 - (ss_res / ss_tot)
print(f"R-squared (goodness of fit): {r_squared:.4f}")

In [56]:
((0.001934641 -- 0.002016714)/52.642) * 1e12

75060882.94517685

# Understand viscosity averaging

In [5]:
import numpy as np

# Read viscosity_output.txt into numpy array ignoring the first line
data = np.loadtxt('visc_output.txt', skiprows=1)

# Reshape the second column to group every 100 values
second_column = data[:, 1][1:]
reshaped = second_column[:].reshape(-1, 100)

# Compute the mean for each group
calculated_averages = reshaped.mean(axis=1)

# Compute the running average
calculated_running_averages = np.cumsum(calculated_averages) / np.arange(1, len(calculated_averages) + 1)

FileNotFoundError: visc_output.txt not found.

In [ ]:
# Read the viscosity_averages.txt file into a numpy array ignoring the first two lines
averages = np.loadtxt('visc_run_avg.txt', skiprows=2)
running_averages = averages[:, 1]

In [ ]:
# Compare the LAMMPS calculated running average with the one we computed
np.allclose(running_averages, calculated_running_averages, atol=1e-6)

True

In [ ]:
running_averages

array([16079. , 16445.2, 17807.3, 19643. , 21473.3, 22648.4, 23349.3,
       23808.7, 23684.4, 24127.5, 24009.8, 24027.2, 24556.8, 24335.6,
       24227.8, 24033.8, 23303.3, 23384.4, 22751.1, 22989.7, 22714.9,
       22870.1, 23244. , 23351.7, 23459. , 23637.7, 23384.5, 23208.6,
       23021.6, 22725.3, 22310.6, 22236.5, 22256.8, 22124.5, 21696.9,
       21601.3, 21806.8, 21569.2, 21515.8, 21487.8, 21481.1, 21410.8,
       21304. , 21230. , 21232.5, 21120.3, 21076.6, 20987.3, 20895.8,
       20922.5])

## Plot viscosity vs. temperature

In [6]:
viscosity_zhou = []
temperatures = [1400, 1550, 1700, 1850, 2000]
xyrate_1_per_s = 1e10

#pair_coeff = "eam/fs"
#potential = "Cu1.eam.fs"
potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Get the last line of the file
    with open(log_file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        # Extract the viscosity value from the last line
        viscosity_value = float(last_line.split()[1])
        viscosity_zhou.append(viscosity_value)
        

In [7]:
viscosity_zhou = []
temperatures = [1400, 1550, 1700, 1850, 2000]
xyrate_1_per_s = 1e10

potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Get the last line of the file
    with open(log_file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        # Extract the viscosity value from the last line
        viscosity_value = float(last_line.split()[1])
        viscosity_zhou.append(viscosity_value)
        

In [ ]:
viscosity_mendelev = []
temperatures = [1400, 1550, 1700, 1850, 2000]
xyrate_1_per_s = 1e10

potential = "Cu1.eam.fs"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Get the last line of the file
    with open(log_file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        # Extract the viscosity value from the last line
        viscosity_value = float(last_line.split()[1])
        viscosity_mendelev.append(viscosity_value)


<class 'str'>


In [28]:
from dfttk.plotly_format import plot_format
import plotly.graph_objects as go

# Create a scatter plot for viscosity vs. temperature
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=temperatures,
    y=viscosity_zhou,
    mode='markers',
    name='Zhou et al. (2004)',
    marker={'size': 10}
))

fig.add_trace(go.Scatter(
    x=temperatures,
    y=viscosity_mendelev,
    mode='markers',
    name='Mendelev et al. (2008)',
    marker={'size': 10}
))

fig.add_vline(x=1358, line_dash='dash', line_color='black')
fig.update_layout(showlegend=True)
fig.update_yaxes(range=[1.5, 6.0])
fig.update_xaxes(range=[1200, 2050])
plot_format(fig, xtitle='Temperature (K)', ytitle='Viscosity (mPa·s)')
fig.show()

ModuleNotFoundError: No module named 'dfttk'

In [34]:
import numpy as np
import plotly.graph_objects as go

viscosity = []
temperatures = [1400]
xyrate_1_per_s = 1e8

#potential = "Cu1.eam.fs"
potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

fig = go.Figure()

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Load data, skipping the first 2 header lines
    data = np.loadtxt(log_file, skiprows=2)
    timesteps = data[:, 0]
    viscosity = data[:, 1]
    fig.add_trace(go.Scatter(
        x=timesteps,
        y=viscosity,
        mode='lines',
        name=f'{temperature} K'
    ))

fig.update_layout(
    title='Viscosity vs. Timestep for Each Temperature (Mendelev et al. 2008)',
    xaxis_title='Timestep (fs)',
    yaxis_title='Viscosity (mPa·s)',
    template='plotly_white',
    showlegend=True
)
fig.show()

## Plot viscosity vs shear strain rate

In [21]:
viscosity_zhou = []
temperature = 1400
xyrate_1_per_s = ['1e6', '1e7', '1e8', '1e9', '1e10']

potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
potential_str = potential.replace('.', '_') 

for strain_rate in xyrate_1_per_s:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{strain_rate}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    
    # Get the last line of the file
    with open(log_file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        
        # Extract the viscosity value from the last line
        viscosity_value = float(last_line.split()[1])
        viscosity_zhou.append(viscosity_value)
        

In [26]:
viscosity_zhou

[89422.4, 71326.2, 0.823171, 4.34229, 3.61727]

In [25]:
from dfttk.plotly_format import plot_format
import plotly.graph_objects as go

# Create a scatter plot for viscosity vs. temperature
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[1e6, 1e7, 1e8, 1e9, 1e10],
    y=viscosity_zhou,
    mode='markers',
    name='Zhou et al. (2004)',
    marker={'size': 10}
))

fig.add_vline(x=1358, line_dash='dash', line_color='black')
fig.update_layout(showlegend=True)
fig.update_yaxes(range=[1.5, 6.0])
fig.update_xaxes(range=[1200, 2050])
plot_format(fig, xtitle='Temperature (K)', ytitle='Viscosity (mPa·s)')
fig.show()